# Import

In [7]:
import numpy as np
from scipy.integrate import quad
import numpy as np
from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin
from sklearn.metrics import zero_one_loss
import sys

sys.path.insert(0, 'python/')
from python.data import make_balanced_dataset, make_unbalanced_dataset
from python.plot import plot_boundary

# Bayes Model

In [8]:

class BayesModel(BaseEstimator, ClassifierMixin):
    def __init__(self):
        pass

    def predict(self, X):
        """Predict class for X.

        Parameters
        ----------
        X : array-like of shape = [n_samples, n_features]
            The input samples.

        Returns
        -------
        y : array of shape = [n_samples]
            The predicted classes, or the predict values.
        """
        # Input validation
        X = np.asarray(X, dtype=np.float)

        if X.ndim != 2:
            raise ValueError("X must be 2 dimensional")
        n_instances, n_features = X.shape

        y = np.ones(n_instances)
        std = 1.6**2
        for i, sample in enumerate(X):
            y[i] = (sample[0] + sample[1] <= -std/3 * np.math.log(0.75/0.25))

        # y = np.array(y > 0.0, dtype=int)

        return y

# Estimate $ E_{x_1, x_2, y}\{\boldsymbol{1}(y\neq h_B(x_1, x_2))\}$ with σ = 1.6

In [9]:


sigma_2 = 1.6**2

lower_bound_positive = (-sigma_2)/3 * np.log(0.75/0.25)
upper_bound_positive = np.inf

lower_bound_negative = -np.inf
upper_bound_negative = (-sigma_2)/3 * np.log(0.75/0.25)

mu_positive = -3
mu_negative = 3

f_positive = lambda t: np.exp(-1/2 * ((t - mu_positive)/(np.sqrt(2*sigma_2)))**2)
f_negative = lambda t: np.exp(-1/2 * ((t - mu_negative)/(np.sqrt(2*sigma_2)))**2)

p_y_0 = 1/(np.sqrt(2*sigma_2) * np.sqrt(2*np.pi)) * quad(f_negative, lower_bound_negative, upper_bound_negative)[0]
p_y_1 = 1/(np.sqrt(2*sigma_2) * np.sqrt(2*np.pi)) * quad(f_positive, lower_bound_positive, upper_bound_positive)[0]

print(p_y_0 * 0.75 + p_y_1 * 0.25)

0.07594122588180544


#  $E_{x_1, x_2, y}\{\boldsymbol{1}(y\neq h_B(x_1, x_2))\}$ computed empirically

In [10]:
X, y = make_unbalanced_dataset(100000, random_state=42)
bayes = BayesModel()

y_pred = bayes.predict(X)
zero_one_loss(y_pred, y)

0.07599999999999996